## **ERPs**

In [34]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

# read data from excel file 
df = pd.read_excel('../../ET_ToDoModerations.xlsx', sheet_name='Tabelle1')

# convert columns to numeric
columns_to_convert = ['MeanEntI']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# choose target columns and set index to subject
df_filtered = df[['Subject','MeanEntI']]
df_filtered.set_index('Subject', inplace=True)
df_filtered.head()

,MeanEntI
Subject,
1,3.333333
3,4.000000
4,6.000000
5,3.000000
8,3.000000


In [136]:
# Load the files
P3_path = "../ERP_Criterion_All/Measures/P3_MA.txt" # P3_MeanAmpl.txt or P3_Latency.txt
N2_path = "../ERP_Criterion_All/Measures/N2_MA.txt" # N2_MeanAmpl.txt or N2_Latency.txt

p3 = pd.read_csv(P3_path, delim_whitespace=True)
n2 = pd.read_csv(N2_path, delim_whitespace=True)

# Extract the subject number from the file name
p3['Subject'] = p3['ERPset'].str.extract('(\d+)$').astype(int)
n2['Subject'] = n2['ERPset'].str.extract('(\d+)$').astype(int)

# Set the subject number as index
p3.set_index('Subject', inplace=True)
n2.set_index('Subject', inplace=True)
p3['bin1_P3']

Subject
1     -7.795005
3     -0.271139
4     -8.611369
5      1.404810
8      7.238353
12     3.942183
15    -7.096182
17    -1.722323
18     9.452059
24    -1.876857
26     0.159383
27     7.508373
28     6.115383
29    -7.149990
32     2.505567
34    -2.984875
35     2.905759
37     0.117564
40     2.866472
42     4.321613
43    -2.716219
45    -5.656874
49    -0.139079
50    10.262437
54     4.156853
56     6.844814
58     8.297737
59     4.140664
60     0.153449
61     1.741144
62    -2.020089
65    -5.305539
66    -6.303784
67     3.222488
68    12.348936
69     4.118261
70     2.363699
72     5.996842
74     6.057314
75     7.835643
76    -3.845332
Name: bin1_P3, dtype: float64

In [137]:
# Merge the dataframes on the subject number
df_P3 = df_filtered.merge(p3, left_on='Subject', right_index=True)
df_N2 = df_filtered.merge(n2, left_on='Subject', right_index=True)

print(df_N2, "\n")
print("Number of subjects =", df_N2.shape[0])

         MeanEntI   bin1_Fp1    bin1_Fz    bin1_F3    bin1_F7   bin1_FT9  \
Subject                                                                    
1        3.333333   2.433341  -2.957755  -6.377655  -5.491745  -8.019588   
3        4.000000  -9.571811  -2.696537   7.731743   6.731492  -4.526723   
4        6.000000  -1.929334  -0.937303  -1.429660   2.242258  -3.585208   
5        3.000000  19.215760   0.591415   9.004481  25.687174  20.427249   
8        3.000000  -3.543144  -0.503322  -2.838248  -8.166782  -7.548464   
12       2.000000   2.491193   5.297017   3.456716   2.349357 -10.316615   
15       5.666667  12.389822   4.705210   7.022247   1.600354 -13.698203   
17       4.833333   7.316833  10.828180  10.658885  -3.803712  -1.232361   
18       2.833333  -4.358080   2.679005   3.878016   0.600425  -1.971930   
24       6.000000  -0.974966  -0.161394  -0.445784  -0.892776  -1.280717   
26       6.000000 -10.905946   4.227328 -11.384440 -28.942746 -27.183792   
27       3.6

In [51]:
# name of the electrodes
electrodos_eeg = [
    "Fp1", "Fz", "F3", "F7", "FT9", "FC5", "FC1", "C3", "T7", "TP9",
    "CP5", "CP1", "Pz", "P3", "P7", "O1", "Oz", "O2", "P4", "P8",
    "TP10", "CP6", "CP2", "Cz", "C4", "T8", "FT10", "FC6", "FC2",
    "F4", "F8", "Fp2"
]

In [138]:
# to obtain p-values of predictors

n_bin = 3 # change this value to get the p-values of the other bins (e.g. 1, 2, 3)
prob_f_statistic_list = list()

for i in range(len(electrodos_eeg)):
    y = df_P3['MeanEntI'] # Change df_N2 to df_P3 to obtain the model for P3
    X = df_P3[f'bin{n_bin}_{electrodos_eeg[i]}'] # Change df_N2 to df_P3 to obtain the model for P3
    X = sm.add_constant(X) # adding a constant

    # fit the model
    model = sm.OLS(y, X)
    results = model.fit()
    
    prob_f_statistic_list.append(results.f_pvalue)

prob_f_statistic_list

[0.6418847180300327,
 0.6277268946893573,
 0.6955120240942068,
 0.7953467647918633,
 0.9018262926297,
 0.939844139544001,
 0.9914629566562345,
 0.2489707225401957,
 0.9301360124047044,
 0.9592089798838925,
 0.24909742995660683,
 0.39384498175374405,
 0.1544655109098658,
 0.01311245747188706,
 0.5321601156117116,
 0.7899379870398981,
 0.9969249649113541,
 0.706381611732888,
 0.5931918595364647,
 0.9174569082027788,
 0.5870556937373388,
 0.06756928012854929,
 0.22443603494854567,
 0.9395162612867182,
 0.7927407449659358,
 0.8078987619421494,
 0.5855999908007499,
 0.7178543871836691,
 0.40132170018455016,
 0.3601337386936744,
 0.4867675237811633,
 0.8668121887243523]

In [140]:
# model for once predictor

y = df_N2['MeanEntI'] # Change df_N2 to df_P3 to obtain the model for P3
X = df_N2[f'bin3_P3'] # Change df_N2 to df_P3 to obtain the model for P3
X = sm.add_constant(X) # adding a constant

# fit the model
model = sm.OLS(y, X)
results = model.fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               MeanEntI   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     7.437
Date:                Sat, 15 Jun 2024   Prob (F-statistic):            0.00953
Time:                        02:53:24   Log-Likelihood:                -64.561
No. Observations:                  41   AIC:                             133.1
Df Residuals:                      39   BIC:                             136.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.6809      0.187     19.637      0.000       3.302       4.060
bin3_P3       -0.1236      0.045     -2.727      0.010      -0.215      -0.032
==============================================================================
Omnibus:                        1.478   Durbin-Watson:                   1.722
Prob(Omnibus):                  0.477   Jarque-Bera (JB):                1.267
Skew:                           0.418   Prob(JB):                        0.531
Kurtosis:                       2.798   Cond. No.                         4.14
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Supplementary Table 1

In [12]:
import pandas as pd
import statsmodels.api as sm

# Cargar el archivo de Excel
file_path = '../28052024_Sqrt_Norm_Table.xlsx'
data = pd.read_excel(file_path)

# Definir la variable dependiente
Y = data['MeanEntPassion']

# Variables independientes corregidas
variables_corrected = ['Sustainable_Cognitive_Effort_Sqrt_Norm', 'FirstMinusLatterRTMT', 'FirstMinusLatterRT',
                       'ET11', 'ET43', 'P3_P3_B1B2_MA', 'N2_P3_B1B2_MA']

# Crear una lista para almacenar los resultados corregidos
results_corrected = []

for var in variables_corrected:
    X = sm.add_constant(data[var])  # Agregar constante para el modelo
    model = sm.OLS(Y, X).fit()
    results_corrected.append({
        'Variable': var,
        'Constant (Beta)': model.params['const'],
        'Constant (SE)': model.bse['const'],
        'Constant (P-value)': model.pvalues['const'],
        'F-statistic': model.fvalue,
        'R-squared': model.rsquared,
        'Beta': model.params[var],
        'Standard Error': model.bse[var],
        'P-value': model.pvalues[var]
    })

# Convertir resultados a DataFrame
results_df_corrected = pd.DataFrame(results_corrected)

results_df_corrected


,Variable,Constant (Beta),Constant (SE),Constant (P-value),F-statistic,R-squared,Beta,Standard Error,P-value
0,Sustainable_Cognitive_Effort_Sqrt_Norm,4.540796,0.230971,7.685383e-22,7.290410,0.157493,0.164512,0.060929,0.010200
1,FirstMinusLatterRTMT,4.569994,0.218325,8.187110e-23,9.295187,0.192466,0.001396,0.000458,0.004114
2,FirstMinusLatterRT,4.533219,0.236465,1.875369e-21,6.536037,0.143535,0.001150,0.000450,0.014578
3,ET11,4.850704,0.221046,1.494131e-23,2.078986,0.050609,1.164227,0.807443,0.157322
4,ET43,4.819564,0.220938,1.852580e-23,1.664476,0.040932,0.934054,0.723991,0.204598
5,P3_P3_B1B2_MA,4.835732,0.219352,1.264990e-23,2.377555,0.057460,-0.076463,0.049589,0.131166
6,N2_P3_B1B2_MA,4.846075,0.212198,3.517539e-24,5.237715,0.118399,-0.117437,0.051314,0.027603


## Covariables Sex, Age and Hand

In [15]:
covariables = ['Sex', 'Age']

results_corrected = []

# Realizar las regresiones con las covariables
for var in variables_corrected:
    # Construir el DataFrame para las variables independientes con las covariables
    X = data[[var] + covariables]
    X = sm.add_constant(X)  # Agregar constante para el modelo
    model = sm.OLS(Y, X).fit()
    
    results_corrected.append({
        'Variable': var,
        'Beta': model.params[var],
        'SE': model.bse[var],
        'P-value': model.pvalues[var],
        'Sex (Beta)': model.params['Sex'],
        'Sex (SE)': model.bse['Sex'],
        'Sex (P-value)': model.pvalues['Sex'],
        'Age (Beta)': model.params['Age'],
        'Age (SE)': model.bse['Age'],
        'Age (P-value)': model.pvalues['Age'],
        'Hand (Beta)': model.params['Hand'],
        'Hand (SE)': model.bse['Hand'],
        'Hand (P-value)': model.pvalues['Hand'],
        'Constant (Beta)': model.params['const'],
        'Constant (SE)': model.bse['const'],
        'Constant (P-value)': model.pvalues['const'],
        'F-statistic': model.fvalue,
        'R-squared': model.rsquared
    })

# Convertir resultados a DataFrame
results_df_corrected = pd.DataFrame(results_corrected)

# Guardar los resultados en un archivo de Excel
output_path = 'Tabla_Regresiones_Con_Covariables2.xlsx'
results_df_corrected.to_excel(output_path, index=False)